<a href="https://colab.research.google.com/github/abursuc/practicals_hti_2019/blob/master/03_dogscats_linear_classifier_dldiy_hti_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a simple neural network from VGG16 features

The last layer of Vgg16 outputs a vector of 1000 categories, because that is the number of categories the competition asked for. Of these categories, some of them certainly correspond to cats and dogs, but at a much more granular level (specific breeds).

We will simply add a Dense layer on top of the imagenet layer, and train the model to map the imagenet classifications of input images of cats and dogs to cat and dog labels.

Note that this is not what we have been doing in the very first lecture!

Have a look at [CS231n: Linear Classification](http://cs231n.github.io/linear-classify/) for more precisions and especially to [CS231n: Softmax classifier](http://cs231n.github.io/linear-classify/#softmax).

## 1. Preparations

In [0]:
!pip install -U bcolz

In case the _Dogs and Cats_ data that we have used in the previous lecture is no longer on your drive you will nee to download it again as shown below:

In [0]:
%mkdir data
%cd /content/data/
!wget http://files.fast.ai/data/dogscats.zip

In [0]:
!unzip dogscats.zip

In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

from tqdm import tqdm
%matplotlib inline

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Using gpu: %s ' % torch.cuda.is_available())

## 2. Data processing

We will first compute the features for all images as the output of the VGG16 network. Unlike previous TP where we were precomputing convolutional features with the goal of fine-tuning the top _fully-connected layers_, here we will take the final output of VGG16, _i.e._ $1000d$ vectors, and use it on a different setting.

In [0]:
# specific functions for loading bcolz files in which the current dataset
# is saved
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
#where you store your features
data_dir = '/content/data/dogscats'

`datasets` is a class of the torchvision package (see `torchvision.datasets`) and deals with data loading. It integrates a multi-threaded loader that fetches images from the disk, groups them in mini-batches and serves them continously to the GPU right after each forward/backward pass through the network.



In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'valid']}

In [0]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}

Depending on the memory of your GPU you can adjust the `batch_size` to be lower or higher.

In [0]:
batch_size = 64
# batch_size = 4


Initialize data loader that will fetch images from disk using num_workers parallel threads.

In [0]:

dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=False, num_workers=4)
                for x in ['train', 'valid']}


Instantiate VGG16 model pretrained on ImageNet from the `torchvision` model Zoo

In [0]:
model_vgg = models.vgg16(pretrained=True)
model_vgg.to(device)

By default all the modules are initialized to train mode (`self.training = True`). Also be aware that some layers have different behavior during train/and evaluation (like `BatchNorm`, `Dropout`) so setting it matters.

Also as a rule of thumb for programming in general, try to explicitly state your intent and set `model.train()` and `model.eval()` when necessary.

In [0]:
model_vgg.eval()

### 2.1 Feature extraction

Function for extracting and storing CNN features, i.e. the ouput of VGG16 model in this case.

In [0]:
def prefeat(dataset):
    features = []
    labels_list = []
    for data in dataset:
        inputs,labels = data
        inputs, labels = inputs.to(device), labels.to(device) 
        
        x = model_vgg(inputs)
        features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    features = np.concatenate([[feat] for feat in features])
    return (features,labels_list)

This should take about 2 minutes 

In [0]:
%%time
feat_train,labels_train = prefeat(dset_loaders['train'])

In [0]:
feat_train.shape


Loading and resizing the images every time we want to use them isn't necessary - instead we should save the processed arrays. By far the fastest way to save and load numpy arrays is using bcolz. This also compresses the arrays, so we save disk space. Here are the functions we'll use to save and load using bcolz (already loaded above...).

In [0]:
%cd /content/data/dogscats/

In [0]:
%mkdir vgg16_1k

In [0]:
save_array(os.path.join(data_dir,'vgg16_1k','feat_1k_train.bc'),feat_train)
save_array(os.path.join(data_dir,'vgg16_1k','labels_1k_train.bc'),labels_train)

In [0]:
%%time
feat_val,labels_val = prefeat(dset_loaders['valid'])

In [0]:
feat_val.shape

In [0]:
save_array(os.path.join(data_dir,'vgg16_1k','feat_1k_val.bc'),feat_val)
save_array(os.path.join(data_dir,'vgg16_1k','labels_1k_val.bc'),lbs_val)

### Optional: uploading precomputed features

This section will allow you to store the precomputed features on your Google drive for later use.

In [0]:
%cd /content/data/dogscats/
!zip -r vgg16_1k vgg16_1k/*

In [0]:

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
upload = drive.CreateFile({'title': 'vgg16_drive.zip'})
upload.SetContentFile('vgg16.zip')
upload.Upload()
print('Uploaded file with ID {}'.format(upload.get('id')))

## 3. Linear model for VGG16 features

We are now ready to define our linear model.

For more details about the [cross entropy cost function](http://neuralnetworksanddeeplearning.com/chap3.html#the_cross-entropy_cost_function)

In [0]:
feat_train = load_array(os.path.join(data_dir,'vgg16_1k','feat_1k_train.bc'))
labels_train = load_array(os.path.join(data_dir,'vgg16_1k','labels_1k_train.bc'))
feat_val = load_array(os.path.join(data_dir,'vgg16_1k','feat_1k_val.bc'))
labels_val = load_array(os.path.join(data_dir,'vgg16_1k','labels_1k_val.bc'))

In [0]:
lm = torch.nn.Sequential(
    torch.nn.Linear(1000, 2),
    torch.nn.LogSoftmax(dim = 1)
)
loss_fn = torch.nn.NLLLoss(reduction='sum')

lm = lm.to(device)

Since our features are currently stacked in a `numpy ndarray`, we need to create a dataset of tensors and then a dataloader.

For the dataset, you can use `torch.from_numpy` and `TensorDataset`

For the dataloader, you should use `torch.utils.data.DataLoader`

In [0]:
batch_size = 128

train_dataset = # your code
test_dataset = # your code
train_loader = torch.utils.data.DataLoader(#your code here)
test_loader = torch.utils.data.DataLoader(#your code here)

### 2.1 Training

We define next a holistic training function (```train_model```) that will:
- run for a pre-defined number of epochs/iterations
- fetch training samples randomly during each epoch(all samples are used during an epoch)
- pass samples through network, compute error, gradients and updates network parameters
- keep and print training statistics: training loss, accuracy


In [0]:
def train_model(model,size,data_loader=None,epochs=1,optimizer=None, loss_fn=None, batch_size=128):
    model.train()
    num_batches = int(size/batch_size)
    loss_t = np.zeros(epochs)
    acc_t = np.zeros(epochs)
    for epoch in range(epochs):
        
        running_loss = 0.0
        running_corrects = 0
        for ii, (inputs,classes) in tqdm(enumerate(data_loader), total=num_batches):
                            
            #
            # your code
            #
            running_loss += # your code
            running_corrects += # your code
            
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        
        loss_t[epoch] = epoch_loss
        acc_t[epoch] = epoch_acc
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    return loss_t, acc_t

In [0]:
def train_model(model,size,data_loader=None,epochs=1,optimizer=None, loss_fn=None, batch_size=128):
    model.train()
    num_batches = int(size/batch_size)

    loss_t = np.zeros(epochs)
    acc_t = np.zeros(epochs)
    for epoch in range(epochs):
        
        running_loss = 0.0
        running_corrects = 0
        for ii, (inputs,classes) in tqdm(enumerate(data_loader), total=num_batches):
        
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs,classes)           
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs.data,1)
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
            
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        
        loss_t[epoch] = epoch_loss
        acc_t[epoch] = epoch_acc
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    return loss_t, acc_t

We set our hyperparameters:
- learning rate
- optimizer to be used for gradient descent, here SGD (Stochastic Gradient Descent)

In [0]:
learning_rate = 1e-4
optimizer_lm = torch.optim.SGD(lm.parameters(), lr=learning_rate)

In [0]:
dset_sizes = {'train': 23000, 'valid': 2000}

We train our model for 100 epochs

In [0]:
%%time
loss1, acc1 = train_model(model=lm,size=dset_sizes['train'],data_loader = train_loader, epochs=100,optimizer=optimizer_lm, loss_fn=loss_fn)

We plot the evolution of the training loss across epochs. 

Ideally is should have a steep descent in the first epochs, then decrease smoothly.

In [0]:
plt.plot(loss1)

We plot the evolution of the accuracy of our model on the training data. The behavior resembles globally to the one of the loss: big improvement at the beginning, then smaller improvements as training advances.


In [0]:
plt.plot(acc1)

The __loss__ helps the network to learn and update the parameters according to the criterion that we give to the network.

The __accuracy__ on the other hand is a performance metric for the task for which we want to use the network for. In many cases the accuracy cannot be integrated as a loss/criterion function, so we need to identify or design loss functions that will guide the model towards the behavior we wish to have for our task.

Next we let the model train for 100 additional epochs

In [0]:
%%time
loss2, acc2 = train_model(model=lm,size=dset_sizes['train'],data_loader =train_loader ,epochs=100,optimizer=optimizer_lm, loss_fn=loss_fn)

Again we plot the loss and accuracy for the current training interval: _epochs[100:200]_.
What changes do you notice? 

In [0]:
plt.plot(loss2)

In [0]:
plt.plot(acc2)

We train the model train for 100 more epochs and plot the evolution of our training indicators. 
How are they evolving comparing to the previous runs?
 

In [0]:
%%time
loss3, acc3 = train_model(model=lm,size=dset_sizes['train'],data_loader =train_loader ,epochs=100,optimizer=optimizer_lm, loss_fn=loss_fn)

In [0]:
plt.plot(loss3)

In [0]:
plt.plot(acc3)

### 2.2 Testing

We define next a holistic test function (```test_model```) that will:
- fetch test samples
- pass samples through network, compute error, accuracy and predictions
- keep and print test statistics: test loss, accuracy


In [0]:
def test_model(model,size,data_loader=None, batch_size=128):
    model.eval()
    num_batches = int(size/batch_size)
    predictions = np.zeros(size)
    running_loss = 0.0
    running_corrects = 0
    count = 0 
    for ii, (inputs,classes) in tqdm(enumerate(data_loader), total=num_batches):
        #
        # your code
        #
        count +=1
        
    print('Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.data.item() / size))
    return predictions, running_loss / size, running_corrects.data.item() / size

We evaluate on the test data a snapshot of our model at _epoch #300_

In [0]:
%%time
preds, loss_val, acc_val = test_model(model=lm,size=dset_sizes['valid'],data_loader=test_loader)

In [0]:
loss_val

## 3. Quantitative analysis

We concatenate the training losses across the 300 training epochs and plot them along with the loss on the test data using a snapshot of our model at epoch #300.

What do you notice? 

In [0]:
plt.plot(np.concatenate((loss1, loss2, loss3)))
plt.plot([loss_val]*300)

We illustrate a similar plot for the training loss values at _epochs[200:300]_

In [0]:
plt.plot(loss3)
plt.plot([loss_val]*100)

We now illustrate the aggregated training accuracies on epochs[0:300] along with the test accuracy for the model at epoch #300.

In [0]:
plt.plot(np.concatenate((acc1, acc2, acc3)))
plt.plot([acc_val]*300)

We train our model for 1000 more epochs.

In [0]:
%%time
loss4, acc4 = train_model(model=lm,size=dset_sizes['train'],data_loader =train_loader ,epochs=1000,optimizer=optimizer_lm)

We test the model snapshot at _epoch #1300_ and keep its statiscs and performance.

In [0]:
%%time
preds2, conf2, loss_val2, acc_val2 = (test_model(model=lm,size=dset_sizes['valid'],feat=feat_val,labels=lbs_val,batch_size=2000))

We aggregate train loss values at _epochs[300:1300]_ and test loss at _epochs[300]_ and _epochs[1300]_.
Do you notice a trend?

In [0]:
plt.plot(np.concatenate((loss3,loss4)))
plt.plot(np.concatenate(([loss_val]*100,[loss_val2]*1000)))

A similar plot for the accuracy values

In [0]:
plt.plot(np.concatenate((acc1, acc2, acc3, acc4)))
plt.plot(np.concatenate(([acc_val]*300,[acc_val2]*1000)))

In [0]:
acc_val

## Exercise

What is happening? 

Make better plots on which we see the evolution of the loss/accuracy on both the training and validation sets as a function of the number of epochs.

## 4. Viewing model prediction (qualitative analysis)

The most important metrics for us to look at are for the validation set, since we want to check for over-fitting.

With our first model we should try to overfit before we start worrying about how to handle that - there's no point even thinking about regularization, data augmentation, etc if you're still under-fitting! (We'll be looking at these techniques after the 2 weeks break...)


As well as looking at the overall metrics, it's also a good idea to look at examples of each of:

   1. A few correct labels at random
   2. A few incorrect labels at random
   3. The most correct labels of each class (ie those with highest probability that are correct)
   4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
   5. The most uncertain labels (ie those with probability closest to 0.5).

In general, these are particularly useful for debugging problems in the model. Since our model is very simple, there may not be too much to learn at this stage...

In [0]:
# Number of images to view for each visualization task
n_view = 8

Selecting correct predictions.

In [0]:
correct = np.where(preds==lbs_val)[0]

In [0]:
from numpy.random import random, permutation
idx = permutation(correct)[:n_view]

In [0]:
idx

In [0]:
def imshow(inp, title=None):
#   Imshow for Tensor.
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'valid']}

dataset_correct = torch.utils.data.DataLoader([dsets['valid'][x] for x in idx],batch_size = n_view,shuffle=True)

In [0]:
for data in dataset_correct:
    inputs_cor,labels_cor = data

In [0]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_cor)

imshow(out, title=[x for x in labels_cor])

In [0]:
from IPython.display import Image, display
for x in idx:
    display(Image(filename=dsets['valid'].imgs[x][0], retina=True))

Selecting incorrect predictions.

In [0]:
incorrect = np.where(preds!=lbs_val)[0]
for x in permutation(incorrect)[:n_view]:
    print(dsets['valid'].imgs[x][1])
    display(Image(filename=dsets['valid'].imgs[x][0], retina=True))

In [0]:
#3. The images we most confident were cats, and are actually cats
correct_cats = np.where((preds==0) & (preds==lbs_val))[0]
most_correct_cats = np.argsort(conf[correct_cats,1])[:n_view]

In [0]:
for x in most_correct_cats:
    display(Image(filename=dsets['valid'].imgs[correct_cats[x]][0], retina=True))

In [0]:
#3. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((preds==1) & (preds==lbs_val))[0]
most_correct_dogs = np.argsort(conf[correct_dogs,0])[:n_view]

In [0]:
for x in most_correct_dogs:
    display(Image(filename=dsets['valid'].imgs[correct_dogs[x]][0], retina=True))

## Exercise

As seen in the first lecture, the last layer of Vgg16 is simply a dense layer that outputs 1000 elements. Therefore, it seems somewhat unreasonable to stack a dense layer meant to find cats and dogs on top of one that's meant to find imagenet categories, in that we're limiting the information available to us by first coercing the neural network to classify to imagenet before cats and dogs...

Instead, do finetuning, i.e remove that last layer and add on a new layer for cats and dogs. 

Compare to what we did in the first lecture.